In [47]:
import torch

def torch_reward_gt(input): #input:(x,u)
    upright = (input[...,2] + 1)/2
    middle = torch.exp(-input[...,0]**2)
    small_ctrl = torch.exp(-4*input[...,-1]**2)
    
    small_ctrl = (4+small_ctrl)/5

    small_vel = torch.exp(-0.5*input[...,4]**2)
    small_vel = (1 + small_vel) / 2

    return (upright*middle*small_ctrl*small_vel).unsqueeze(-1)

def torch_reward_walker(input): #input:(x,u)
    upright = (torch.cos(input[...,1]) + 1)/2
    #standing = torch.exp(-16*(input[...,0])**2)
    standing = torch.clip(1-1.0*torch.abs(input[...,0]),0,1)
    standing_reward = (3*standing + upright)/4

    #move_reward = torch.exp(-8*(1.0-input[...,8])**2)
    move_reward = torch.clip(input[...,8]/1.0,0.0,1)

    return (standing_reward*move_reward).unsqueeze(-1)


In [48]:
data = torch.load("../Data/Human_fb/Cartpole_4/history_data.pt")
label = torch.load("../Data/Human_fb/Cartpole_4/history_label_tensor.pt")

print(data.shape)
print(label.shape)

torch.Size([11, 10, 2, 50, 6])
torch.Size([11, 10])


/tmp/ipykernel_1099681/414897891.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("../Data/Human_fb/Cartpole_4/history_data.pt")
/tmp/ipykernel_1099681/

In [49]:
rewards = torch_reward_gt(data).squeeze(-1).sum(-1)
print(rewards.shape)

torch.Size([11, 10, 2])


In [50]:
label_true = rewards[...,0] > rewards[...,1]
label_true

tensor([[False, False,  True, False,  True,  True, False,  True,  True,  True],
        [False, False, False, False,  True, False, False,  True, False, False],
        [False,  True,  True,  True, False,  True, False,  True,  True,  True],
        [False, False, False,  True,  True, False, False, False,  True,  True],
        [False,  True,  True,  True,  True, False, False, False, False,  True],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [False,  True,  True, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True, False,  True,  True],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False,  True,  True,  True, False,  True,  True,  True, False],
        [False, False, False, False,  True, False, False, False, False, False]])

In [51]:
label.bool() == label_true

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True, False,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True, False,  True,  True,  True, False,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True, False,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [39]:
data = torch.load("../Data/Human_fb/Walker_4/history_data.pt")
label = torch.load("../Data/Human_fb/Walker_4/history_label_tensor.pt")

print(data.shape)
print(label.shape)

torch.Size([21, 10, 2, 50, 23])
torch.Size([21, 10])


/tmp/ipykernel_1099681/2684172697.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("../Data/Human_fb/Walker_4/history_data.pt")
/tmp/ipykernel_1099681/2

In [40]:
rewards = torch_reward_walker(data).squeeze(-1).sum(-1)
print(rewards.shape)

torch.Size([21, 10, 2])


In [41]:
label_true = rewards[...,0] > rewards[...,1]
label_true

tensor([[False, False, False, False,  True, False, False,  True,  True, False],
        [False, False,  True, False,  True, False, False, False,  True, False],
        [ True,  True,  True, False, False, False,  True, False,  True, False],
        [False, False,  True, False, False,  True, False, False, False, False],
        [ True,  True, False, False,  True,  True,  True,  True, False,  True],
        [ True,  True, False,  True,  True, False, False, False,  True, False],
        [False, False, False, False, False, False, False,  True, False,  True],
        [False, False, False, False, False,  True,  True, False, False,  True],
        [False, False, False,  True, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False,  True, False],
        [ True, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True, False,  True,  Tr

In [42]:
label.bool() == label_true

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  Tr